In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
gdf_building = gpd.read_file('../../data/processed/building/building_sample.geojson')

In [3]:
gdf_building.head()

,height,mpluto_bbl,cnstrct_yr,globalid,bin,shape_len,shape_area,office_area,retail_area,residential_area,...,idw_aadt_2021,idw_atvc_2018,idw_atvc_2019,idw_atvc_2020,office_within_450ft,retail_within_450ft,residential_within_450ft,distance_to_park,distance_to_school,geometry
0,55.89,1006100054,1890,{3AEADBE0-CC54-4D24-B238-035EACC2FCA7},1010689,0.0,636520502.758,0,1050,8294,...,8350.144642,123.350334,147.058962,304.025386,2120,52008,312275,74.862704,425.271876,"MULTIPOLYGON (((984248.206 206864.602, 984228...."
1,43.48,1014390003,1920,{16E3A900-8D55-4377-ABC9-3CFEAD5AABEA},1044690,None,636520502.758,0,2175,6525,...,13486.718605,151.540038,189.112186,239.713791,9539,49216,1197236,787.992426,609.659406,POINT (994870.595 217574.392)
2,164.86,1011480001,1925,{A569DF64-C060-4E22-A45B-C3AAF3994CF0},1030169,0.0,636520502.758,0,5000,118840,...,15411.109097,126.416875,155.399025,245.756263,2205,52578,438942,162.862776,338.724963,"MULTIPOLYGON (((989996.502 223961.988, 989991...."
3,57.41,1019660033,1901,{FF1D608A-A8F9-4BED-8330-01D6E785BBDE},1084099,0.0,636520502.758,0,3500,15130,...,14449.074278,100.988607,140.355271,284.405716,35299,23925,795466,244.292302,609.863267,"MULTIPOLYGON (((996485.007 235709.202, 996487...."
4,80.0,1019880018,2017,{F3EEBE92-794C-4F0B-914A-86A784D1532B},1089415,0.0,636520502.758,2962,0,16905,...,15793.074614,88.857910,146.777034,285.662061,2962,23684,68622,431.172222,558.570996,"MULTIPOLYGON (((996978.357 237617.591, 997012...."


In [4]:
gdf_building.columns

Index(['height', 'mpluto_bbl', 'cnstrct_yr', 'globalid', 'bin', 'shape_len',
       'shape_area', 'office_area', 'retail_area', 'residential_area',
       'OBJECTID', 'street_width_min', 'street_width_max', 'posted_speed',
       'betweeness', 'station_complex_id', 'ridership_evening',
       'ridership_late_night', 'ridership_midday', 'ridership_morning',
       'ridership_night', 'distance_from_station(ft)', 'idw_aadt_2019',
       'idw_aadt_2020', 'idw_aadt_2021', 'idw_atvc_2018', 'idw_atvc_2019',
       'idw_atvc_2020', 'office_within_450ft', 'retail_within_450ft',
       'residential_within_450ft', 'distance_to_park', 'distance_to_school',
       'geometry'],
      dtype='object')

In [5]:
df_restaurant = pd.read_csv('../../data/processed/restaurants_coord.csv')
points = [Point(x, y) for x, y in zip(df_restaurant['longitude'], df_restaurant['latitude'])]
gdf_restaurants = gpd.GeoDataFrame(df_restaurant, geometry=points, crs=4326).to_crs(gdf_building.crs)

In [6]:
def count_restaurants_within_buffer(building_geometry, buffer_distance=328):
    '''
    input: building_geometry (shapely Polygon or MultiPolygon), buffer_distance (float)
    output: restaurant_count (int)
    '''
    # create a buffer zone around the building
    buffer_zone = building_geometry.buffer(buffer_distance)
    # find all restaurants that are within the buffer zone
    intersecting_restaurants = gdf_restaurants.loc[gdf_restaurants.within(buffer_zone)]
    # count the number of restaurants within the buffer zone
    restaurant_count = len(intersecting_restaurants)
    return restaurant_count

In [7]:
gdf_building.loc[:,'food_100'] = gdf_building['geometry'].apply(count_restaurants_within_buffer, buffer_distance=328)
gdf_building.loc[:,'food_400'] = gdf_building['geometry'].apply(count_restaurants_within_buffer, buffer_distance=1312)
gdf_building.loc[:,'food_800'] = gdf_building['geometry'].apply(count_restaurants_within_buffer, buffer_distance=2624)
gdf_building.loc[:,'food_1000'] = gdf_building['geometry'].apply(count_restaurants_within_buffer, buffer_distance=3280)

In [8]:
# calculate the mean of the AADT and ATVC values
gdf_building.loc[:,'idw_aadt_mean'] = gdf_building.loc[:,['idw_aadt_2019', 'idw_aadt_2020', 'idw_aadt_2021']].mean(axis=1)
gdf_building.loc[:,'idw_atvc_mean'] = gdf_building.loc[:,['idw_atvc_2018','idw_atvc_2019', 'idw_atvc_2020']].mean(axis=1)

In [9]:
# drop unnecessary columns and save the GeoDataFrame to a GeoJSON file
gdf_building.to_crs(4326).to_file('../../data/processed/building/building_sample.geojson', driver='GeoJSON')